In [295]:
import numpy as np
import math
import matplotlib.pyplot as plt

In [296]:
def splitData(data, ratio):
    np.random.shuffle(data)
    train_size = int(len(data) * ratio)
    train_set = data[:train_size]
    test_set = data[train_size:]
    return train_set, test_set

In [297]:
def timeToMinutes(time):
    splitted = time.split(":")
    if int(splitted[0]) > 23 or int(splitted[1]) > 59:
        raise ValueError
    return int(splitted[0]) * 60 + int(splitted[1])

In [298]:
def processTime(time):
    minutes = timeToMinutes(time)
    return  math.sin(minutes / (24 * 60)*math.pi)

In [299]:
def loadData(codesToIndexes):
    data = []
    for i in range(1, 70):
        filePrefix = 'Diabetes-Data/data-'
        if i < 10:
            filePrefix += '0'
        fileHandle = open(filePrefix+str(i), 'r')
        for line in fileHandle:
            line = line.strip()
            if line:
                try:
                    parts = line.split('\t')
                    time = processTime(parts[1])
                    code = int(parts[2])
                    if code not in codesToIndexes.keys():
                        raise ValueError
                    number = int(parts[3])
                    data.append([time, code, number])
                except ValueError:
                    pass
                except IndexError:
                    pass
        
        fileHandle.close()
    return data


In [300]:
def encodeOutput(Y,codesToIndexes):
    codesToIndexes = {33:0, 34:1, 35:2, 48:3, 57:4, 58:5, 59:6, 60:7, 61:8, 62:9, 63:10, 64:11, 65:12, 66:13, 67:14, 68:15, 69:16, 70:17, 71:18, 72:19}
    Y_encoded = np.zeros((Y.shape[0], 20))
    for i, code in enumerate(Y):
        index = codesToIndexes[code]
        Y_encoded[i, index] = 1
    return Y_encoded

In [301]:
codesToIndexes = {33:0, 34:1, 35:2, 48:3, 57:4, 58:5, 59:6, 60:7, 61:8, 62:9, 63:10, 64:11, 65:12, 66:13, 67:14, 68:15, 69:16, 70:17, 71:18, 72:19}
data = np.array(loadData(codesToIndexes))
data, test = splitData(data, 0.8)
X = data[:, [0, 2]]
X_test = test[:, [0, 2]]
Y = data[:, 1]
Y_test = test[:, 1]
Y_encoded = encodeOutput(Y,codesToIndexes)
Y_test_encoded = encodeOutput(Y_test,codesToIndexes)

In [314]:
def initializeParameters(nX, nH, nY, numberOfHiddenLayers=1):
    parameters = {}
    if numberOfHiddenLayers < 1:
        raise ValueError   
    parameters["W1"] = np.random.randn(nX, nH)
    parameters["b1"] = np.zeros((1,nH))
    if numberOfHiddenLayers > 1:
        for i in range(2, numberOfHiddenLayers):
            parameters["W"+str(i+1)] = np.random.randn(nH[i-2], nH[i-1])
            parameters["b"+str(i+1)] = np.zeros((1,nH[i-1]))
    parameters["W"+str(numberOfHiddenLayers+1)] = np.random.randn(nH, nY)
    W1 = np.random.randn(nX, nH)
    b1 = np.zeros((1,nH))
    b1 = np.random.randn(1,nH)
    W2 = np.random.randn(nH,nY)
    b2 = np.zeros((1,nY))
    b2 = np.random.randn(1,nY)
    parameters = {"W1": W1, "b1": b1, "W2": W2, "b2": b2}
    return parameters

### Architektura sieci neuronowej

- **Warstwa wejściowa:** N neurony (wejścia)
- **Warstwy ukryte:** L warstw, każda z dowolną liczbą neuronów
- **Warstwa wyjściowa:** K neuronów (klasyfikacja na K kategorii)

### Funkcje aktywacji

- **Warstwy ukryte:** Sigmoid
  - $$\sigma(z) = \frac{1}{1 + e^{-z}}$$
- **Warstwa wyjściowa:** Softmax
  - $$\text{Softmax}(z_i) = \frac{e^{z_i}}{\sum_{j=1}^{K} e^{z_j}}$$

### Propagacja w przód

Dla każdej warstwy l (od 1 do L+1, gdzie L+1 to warstwa wyjściowa):

1. **Sygnał wejściowy:** $$Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]}$$
2. **Aktywacja:** 
   - Dla warstw ukrytych: $$A^{[l]} = \sigma(Z^{[l]})$$
   - Dla warstwy wyjściowej: $$A^{[L+1]} = \text{Softmax}(Z^{[L+1]})$$

### Wsteczna propagacja błędu

1. **Błąd na wyjściu (warstwa L+1):**
   - $$\delta^{[L+1]} = A^{[L+1]} - Y$$

Dla każdej warstwy l od L do 1:

2. **Błąd dla warstwy l:**
   - $$\delta^{[l]} = (W^{[l+1]T} \delta^{[l+1]}) \odot \sigma'(Z^{[l]})$$
3. **Gradient dla wag i biasów:**
   - $$\nabla W^{[l]} = \delta^{[l]} A^{[l-1]T}$$
   - $$\nabla b^{[l]} = \sum(\delta^{[l]}, \text{axis} = 0)$$

### Aktualizacja wag

- Wagi i biasy są aktualizowane za pomocą metody spadku gradientu:
  - $$W^{[l]} = W^{[l]} - \alpha \nabla W^{[l]}$$
  - $$b^{[l]} = b^{[l]} - \alpha \nabla b^{[l]}$$


In [303]:
def costFunction(Y, Y_hat):
    return - np.sum(Y * np.log(Y_hat))

In [304]:
def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

In [305]:
def softmax(Z):
    return np.exp(Z) / np.sum(np.exp(Z), axis=1, keepdims=True)

In [306]:
def propagate(X, Y, parameters):
    Z1 = np.dot(X, parameters["W1"]) + parameters["b1"]
    A1 = sigmoid(Z1)
    Z2 = np.dot(A1, parameters["W2"]) + parameters["b2"]
    A2 = softmax(Z2)
    cost = costFunction(Y, A2)
    # pochodne funkcji straty:
    dA2 = A2 - Y
    dW2 = np.dot(A1.T,dA2) 
    db2 = np.sum(dA2, axis=0,keepdims=True)
    dA1 = np.dot(dA2, parameters["W2"].T)
    dZ1 = dA1 * (A1 * (1 - A1))
    dW1 = np.dot(X.T, dZ1)
    db1 = np.sum(dZ1, axis=0)
    gradients = {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}
    return gradients, cost

In [307]:
def updateParameters(parameters, gradients, learningRate):
    parameters["W1"] = parameters["W1"] - learningRate * gradients["dW1"]
    parameters["b1"] = parameters["b1"] - learningRate * gradients["db1"]
    parameters["W2"] = parameters["W2"] - learningRate * gradients["dW2"]
    parameters["b2"] = parameters["b2"] - learningRate * gradients["db2"]
    return parameters

In [308]:
def optimize(X, Y, parameters, learningRate, numberOfIterations, printCost=False):
    costs = []
    for i in range(numberOfIterations):
        gradients, cost = propagate(X, Y, parameters)
        parameters = updateParameters(parameters, gradients, learningRate)
        costs.append(cost)
        if i % 100 == 0 and printCost:
            print("Cost after iteration {}: {}".format(i, cost))
    return parameters, costs

In [309]:
def predict(X, parameters):
    Z1 = np.dot(X, parameters["W1"]) + parameters["b1"]
    A1 = sigmoid(Z1)
    Z2 = np.dot(A1, parameters["W2"]) + parameters["b2"]
    A2 = softmax(Z2)
    return np.argmax(A2, axis=1)

In [310]:
parameters = initializeParameters(2, 15, 20)
params, costs = optimize(X,Y_encoded, parameters, 0.00001, 5000, True)

Cost after iteration 0: 132400.40176590235
Cost after iteration 100: 42764.27812233248


/tmp/ipykernel_10313/70635953.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-Z))


Cost after iteration 200: 39962.974077592895
Cost after iteration 300: 37948.635131375704
Cost after iteration 400: 37190.733709314525
Cost after iteration 500: 36863.885935666556
Cost after iteration 600: 33642.342378776
Cost after iteration 700: 34388.07751649178
Cost after iteration 800: 34646.1160511067
Cost after iteration 900: 33530.34789254052
Cost after iteration 1000: 31029.99481889547
Cost after iteration 1100: 30574.590681991154
Cost after iteration 1200: 30277.59696615229
Cost after iteration 1300: 30774.44383228022
Cost after iteration 1400: 31064.170353721638
Cost after iteration 1500: 30838.16485950695
Cost after iteration 1600: 30521.0427235533
Cost after iteration 1700: 30163.701314547125
Cost after iteration 1800: 29937.146245432494
Cost after iteration 1900: 29743.48213480076
Cost after iteration 2000: 29608.058231934298
Cost after iteration 2100: 29487.54784519617
Cost after iteration 2200: 29338.418256139932
Cost after iteration 2300: 29203.97650582283
Cost after i

In [311]:
a = predict(X, params)
i=18
print(a[i])
sum =0
timesGuessedA = {}
timesGuessedY = {}
trafione = {}
for i in range(a.shape[0]):
    timesGuessedA[a[i]] = timesGuessedA.get(a[i], 0) + 1    
    timesGuessedY[np.argmax(Y_encoded[i])] = timesGuessedY.get(np.argmax(Y_encoded[i]), 0) + 1
    if a[i] == np.argmax(Y_encoded[i]):
        trafione[a[i]] = trafione.get(a[i], 0) + 1
        sum += 1

print(sum,a.shape[0])
print(timesGuessedA)
print(timesGuessedY)
print(trafione)
print(sum/a.shape[0]*100)

/tmp/ipykernel_10313/70635953.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-Z))


5
11677 22988
{0: 8203, 5: 10120, 1: 3045, 12: 679, 7: 334, 9: 281, 14: 326}
{0: 7531, 7: 2199, 3: 1482, 5: 2776, 1: 2946, 13: 123, 11: 719, 4: 797, 9: 2487, 10: 172, 2: 820, 14: 262, 8: 55, 18: 79, 19: 78, 12: 264, 16: 55, 6: 16, 17: 105, 15: 22}
{0: 6801, 5: 2698, 1: 1760, 7: 80, 9: 65, 14: 91, 12: 182}
50.7960675134853


In [312]:
a = predict(X_test, params)
i=18
print(a[i])
sum =0
timesGuessedA = {}
timesGuessedY = {}
trafione = {}
for i in range(a.shape[0]):
    timesGuessedA[a[i]] = timesGuessedA.get(a[i], 0) + 1    
    timesGuessedY[np.argmax(Y_test_encoded[i])] = timesGuessedY.get(np.argmax(Y_test_encoded[i]), 0) + 1
    if a[i] == np.argmax(Y_test_encoded[i]):
        trafione[a[i]] = trafione.get(a[i], 0) + 1
        sum += 1

print(sum,a.shape[0])
print(timesGuessedA)
print(timesGuessedY)
print(trafione)
print(sum/a.shape[0]*100)

0
2863 5747
{5: 2579, 1: 729, 0: 2041, 7: 78, 12: 164, 9: 62, 14: 94}
{7: 559, 2: 233, 0: 1815, 5: 703, 3: 377, 10: 47, 1: 734, 9: 631, 8: 11, 4: 192, 15: 12, 11: 185, 17: 34, 14: 64, 13: 31, 12: 67, 19: 16, 16: 13, 18: 19, 6: 4}
{0: 1666, 5: 684, 1: 424, 7: 20, 9: 11, 12: 39, 14: 19}
49.817295980511574


/tmp/ipykernel_10313/70635953.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-Z))


In [313]:
# plt.plot(costs[0:100])